The goal is to create embeddings for 2 different type of docs and converse between them

In [54]:
!python create_web_embeddings.py

The above code was used to create embeddings for a job post that i randomly found

In [36]:
from dotenv import load_dotenv
import os
import pickle
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [39]:
#testing that embedding
from langchain import FAISS
embeddings = OpenAIEmbeddings()
VectorStore = FAISS.load_local("website_embeddings",embeddings)

llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VectorStore.as_retriever())

print(chain({"question": "what is the job name?"}, return_only_outputs=True))

{'answer': 'The job name is US-Operations Expert or Operations Expert.\n', 'sources': 'https://jobs.apple.com/en-us/details/114438152/us-operations-expert?team=APPST'}


In [55]:
!python create_pdf_embeddings.py

In [45]:
VectorStore = FAISS.load_local("pdf_embeddings",embeddings)

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VectorStore.as_retriever())

print(chain({"question": "what are the skills listed?"}, return_only_outputs=True))

{'answer': 'The technical skills listed are Python OpenCV, TensorFlow, MobileNets, NumPy, Pandas, Matplotlib, Seaborn, Plotly and Cu\x0finks, Scikit-Learn, Microsoft Azure, Python, Java, C, C++, MySQL, Intellij Environment, Visual Studio, MacOS, Linux & Windows. The soft skills listed are Highly motivated to learn about new tools and technologies, Ability to Work Under Pressure, Detail-Oriented, Critical Thinker, Problem Solver, Self-Starter, Great at Time Management, Adaptability, Great Interpersonal Skills.\n', 'sources': 'resume.pdf'}


In [56]:
#merging two vectors:
VectorStore1 = FAISS.load_local("pdf_embeddings",embeddings)
VectorStore2 = FAISS.load_local("website_embeddings",embeddings)
VectorStore1.merge_from(VectorStore2)

In [57]:
VectorStore1.docstore._dict


{'8ace0599-32be-4953-adb9-54305c78bdc7': Document(page_content='ARYAN S PILLAI\nVellore Institute of Technology, Vellore\nCSE WITH SPECIALIZATION IN BUSINESS SYSTEMS\nhttps://github.com/aryan619348 \x05aryansantosh.pillai2020@vitstudent.ac.in\nEXPERIENCE\nData Analytics using Deep Learning at NUS (December,2022)\nCompleted a comprehensive course related to data analytics and deep learning under reputed professors\nat NUS.\nDeep Learning with HPE (December,2022)\nCompleted HPE seminars for azure deployment and deployed a live project on azure cloud.\nUsing AI Models for Market Segmentation(Aug,2022-Sep,2022)\nResearched AI techniques that are currently being used to conduct market segmentation by businesses\nand compared algorithms including Biclustering, SKC, Kmeans algorithms to see how the results di\x0ber\ndevelop a better algorithm.\nPROJECTS\nStock Market Price Prediction\nCreated a LSTM model and a CNN-LSTM model to predict the future prices of NIFTY stock index\nand deployed the

In [60]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VectorStore1.as_retriever())
print(chain({"question": "what are the job requirements? and does aryan pillai have them?"}, return_only_outputs=True))

{'answer': 'For the first three content sections, there is no relevant text to answer the question about job requirements or whether Aryan Pillai has them. For the last content section, the job requirements include managing and meeting multiple inventory deadlines each week, observing guidelines for secure access to products and movement through the stockroom, and being flexible with work hours based on business needs. It is not mentioned whether Aryan Pillai has these requirements.\n', 'sources': 'resume.pdf, https://jobs.apple.com/en-us/details/114438152/us-operations-expert?team=APPST'}


The result is not what i wanted at all 😞 

lets try creating the embeddings together.

In [4]:
!python create_multiplefiletype_embeddings.py

In [10]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

import pickle

with open("multiple_file_embeddings.pkl", "rb") as f:
    VectorStore = pickle.load(f)

llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VectorStore.as_retriever())

print(chain({"question": "Tell me skill possesed by aryan"}, return_only_outputs=True))

{'answer': 'Aryan possesses skills in data analytics, deep learning, AI techniques, LSTM modeling, CNN-LSTM modeling, TensorFlow, MobileNets, Keras, OpenCV, GAN, TensorFlow.js, and image augmentation.\n', 'sources': 'resume.pdf'}


In [12]:
print(chain({"question": "Tell me skills needed for the job posted by apple"}, return_only_outputs=True))

{'answer': 'Skills needed for the job posted by Apple include the ability to think quickly and perform problem-solving tasks, leadership skills, strong organizational skills, ability to manage and meet multiple inventory deadlines, willingness to observe guidelines for secure access to products and movement through the stockroom, and flexibility with work schedule.\n', 'sources': 'https://jobs.apple.com/en-us/details/114438152/us-operations-expert?team=APPST'}


In [13]:
print(chain({"question": "Does aryan have the necessary skills for the apple job?"}, return_only_outputs=True))

{'answer': "I don't know. \n", 'sources': 'None.'}


WELL THAT DOES NOT WORK TOO  😞 

LETS TRY A CHAIN WITH MEMORY? maybe that works????

In [30]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain import OpenAI

llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')

chain = load_qa_chain(llm=llm, chain_type="stuff")


In [33]:
with open("multiple_file_embeddings.pkl", "rb") as f:
    docsearch = pickle.load(f)

query="What is are aryans skills?"
docs = docsearch.similarity_search(query)

print(chain.run(input_documents=docs, question=query))


Aryan's technical skills include Python OpenCV, TensorFlow, MobileNets, NumPy, Pandas, Matplotlib, Seaborn, Plotly, Cuinks, Scikit-Learn, Microsoft Azure, MySQL, Python, Java, C, C++, Intellij Environment, Visual Studio, MacOS, Linux & Windows. He also has certifications in Data Analytics using Deep Learning, Hewlett Packard Enterprise Education Services, Deep Learning, and Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning. 

In terms of soft skills, Aryan is highly motivated to learn about new tools and technologies, able to work under pressure, detail-oriented, critical thinker, problem solver, self-starter, great at time management, adaptable, and has great interpersonal skills.


In [34]:
query="What is are skills needed for the job?"
docs = docsearch.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

The Operations Expert job requires strong organizational skills, the ability to think quickly and perform problem-solving tasks, and leadership skills, whether guiding by example or coaching a group. Additionally, the job requires the ability to manage and meet multiple inventory deadlines each week, flexibility with your schedule, and the willingness to observe guidelines to allow secure access to products and movement through the stockroom.


In [35]:
query="Does aryan have the skills needed for the job?"
docs = docsearch.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

Based on the information provided, it is not possible to determine if Aryan has the skills needed for the job as the job requirements and qualifications are not specified.


In [36]:
query="Does aryan have the skills needed for the Operations Expert job?"
docs = docsearch.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

Based on the context provided, there is no information about whether Aryan has the skills needed for the Operations Expert job at Apple Retail. The context only provides information about Aryan's education, experience, projects, technical skills, certifications, and soft skills.
